# Project Overview

Student provides a high-level overview of the project in layman’s terms. Background information such as the problem domain, the project origin, and related data sets or input data is given.

# Problem Statement

The problem which needs to be solved is clearly defined. A strategy for solving the problem, including discussion of the expected solution, has been made.

# Metrics

Metrics used to measure performance of a model or result are clearly defined. Metrics are justified based on the characteristics of the problem.

# Data Exploration

If a dataset is present, features and calculated statistics relevant to the problem have been reported and discussed, along with a sampling of the data. In lieu of a dataset, a thorough description of the input space or input data has been made. Abnormalities or characteristics about the data or input that need to be addressed have been identified.

# Exploratory Visualization

A visualization has been provided that summarizes or extracts a relevant characteristic or feature about the dataset or input data with thorough discussion. Visual cues are clearly defined.

# Algorithms and Techniques

Algorithms and techniques used in the project are thoroughly discussed and properly justified based on the characteristics of the problem.

# Benchmark

Student clearly defines a benchmark result or threshold for comparing performances of solutions obtained.

# Data Preprocessing

All preprocessing steps have been clearly documented. Abnormalities or characteristics about the data or input that needed to be addressed have been corrected. If no data preprocessing is necessary, it has been clearly justified.

# Implementation

The process for which metrics, algorithms, and techniques were implemented with the given datasets or input data has been thoroughly documented. Complications that occurred during the coding process are discussed.

# Refinement

The process of improving upon the algorithms and techniques used is clearly documented. Both the initial and final solutions are reported, along with intermediate solutions, if necessary.

# Model Evaluation and Validation

The final model’s qualities — such as parameters — are evaluated in detail. Some type of analysis is used to validate the robustness of the model’s solution.

# Justification

The final results are compared to the benchmark result or threshold with some type of statistical analysis. Justification is made as to whether the final model and solution is significant enough to have adequately solved the problem.

# Free-Form Visualization

A visualization has been provided that emphasizes an important quality about the project with thorough discussion. Visual cues are clearly defined.

# Reflection

Student adequately summarizes the end-to-end problem solution and discusses one or two particular aspects of the project they found interesting or difficult.

# Improvement

Discussion is made as to how one aspect of the implementation could be improved. Potential solutions resulting from these improvements are considered and compared/contrasted to the current solution.

# Presentation

Project report follows a well-organized structure and would be readily understood by its intended audience. Each section is written in a clear, concise and specific manner. Few grammatical and spelling mistakes are present. All resources used to complete the project are cited and referenced.

# Functionality

Code is formatted neatly with comments that effectively explain complex implementations. Output produces similar results and solutions as to those discussed in the project.

# Use Tensforflow build BCNN 

## What is BCNN?

BCNN is a recignition architecture that consists of two feature extrators whose outputs are multipled using outer product at each location of the image and pooled to obtaine an image descriptor(as cited in http://vis-www.cs.umass.edu/bcnn/docs/bcnn_iccv15.pdf).

So basically we need to use two feature extractors, namely A and B, and use the outer product of A and B as the input of FC-layer, the whole architecture as below.


![BCNN architecture](img/bcnn.png)

## Implement

In [1]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
#from sklearn.utils import shuffle
#import tflearn
#from tflearn.data_preprocessing import ImagePreprocessing
#from tflearn.data_augmentation import ImageAugmentation
#import os
from tflearn.data_utils import shuffle

import pickle 
#from tflearn.data_utils import image_preloader
import h5py
import math
import logging
import random
import time

C:\Users\mingcheng.yen\AppData\Local\Continuum\anaconda3\envs\python3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


curses is not supported on this machine (please install/reinstall curses for an optimal experience)


## Load data

In [2]:
train_data = h5py.File('train_112.h5', 'r')
val_data = h5py.File('val_112.h5', 'r')
X_train, Y_train = train_data['X'], train_data['Y']
X_val, Y_val = val_data['X'], val_data['Y']
test_data = h5py.File('test_112.h5', 'r')
X_test, Y_test = test_data['X'], test_data['Y']
print("Data shapes -- (train, val, test)", X_train.shape, X_val.shape, X_test.shape)


Data shapes -- (train, val, test) (506, 112, 112, 3) (203, 112, 112, 3) (203, 112, 112, 3)


In [3]:
def random_flip_right_to_left(image_batch):
    result = []
    for n in range(image_batch.shape[0]):
        if bool(random.getrandbits(1)):
            result.append(image_batch[n][:,::-1,:])
        else:
            result.append(image_batch[n])
    return result

## VGG16

In [4]:
class vgg16_train_last:
    
    def __init__(self, imgs ,weights=None, sess=None):
        self.images=imgs
        #self.imgs=imgs
        self.last_layer_parameters=[]
        self.parameters=[]
        self.cnn_layers()
        self.fc_layers()
        self.weight_file=weights
        #self.load_weights(weights, sess)
    
    def cnn_layers(self):
        
        
        #conv1_1
        with tf.name_scope('conv1_1') as scope:
            weights = tf.Variable(tf.truncated_normal([3, 3, 3, 64], dtype=tf.float32,
                                                     stddev=1e-1), trainable=False)
            biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                                   trainable=False)
            conv = tf.nn.conv2d(self.images, weights, [1, 1, 1, 1], padding='SAME')
            out = tf.nn.bias_add(conv, biases)
            self.conv1_1 = tf.nn.relu(out, name=scope)
            self.parameters += [weights, biases]
            
        # conv1_2
        with tf.name_scope('conv1_2') as scope:
            weights = tf.Variable(tf.truncated_normal([3, 3, 64, 64], dtype=tf.float32,
                                                     stddev=1e-1), trainable=False)
            biases = tf.Variable(tf.constant(0.0, shape=[64],  dtype=tf.float32),
                                   trainable=False)
            conv = tf.nn.conv2d(self.conv1_1, weights, [1, 1, 1, 1], padding='SAME')
            out = tf.nn.bias_add(conv, biases)
            self.conv1_2 = tf.nn.relu(out, name=scope)
            self.parameters += [weights, biases]
    
        # pool1
        self.pool1 = tf.nn.max_pool(self.conv1_2,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool1')
        
        # conv2_1
        with tf.name_scope('conv2_1') as scope:
            weights = tf.Variable(tf.truncated_normal([3, 3, 64, 128], dtype=tf.float32,
                                                     stddev=1e-1), trainable=False)
            biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),
                                   trainable=False)
            conv = tf.nn.conv2d(self.pool1, weights, [1, 1, 1, 1], padding='SAME')

            out = tf.nn.bias_add(conv, biases)
            self.conv2_1 = tf.nn.relu(out, name=scope)
            self.parameters += [weights, biases]
            
        # conv2_2
        with tf.name_scope('conv2_2') as scope:
            weights = tf.Variable(tf.truncated_normal([3, 3, 128, 128], dtype=tf.float32,
                                                     stddev=1e-1), trainable=False)
            biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32), trainable=False)
            conv = tf.nn.conv2d(self.conv2_1, weights, [1, 1, 1, 1], padding='SAME')
            
            out = tf.nn.bias_add(conv, biases)
            self.conv2_2 = tf.nn.relu(out, name=scope)
            self.parameters += [weights, biases]
        
        # pool2
        self.pool2 = tf.nn.max_pool(self.conv2_2,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool2')
        
        # conv3_1
        with tf.name_scope('conv3_1') as scope:
            weights = tf.Variable(tf.truncated_normal([3, 3, 128, 256], dtype=tf.float32,
                                                     stddev=1e-1),  trainable=False)
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                   trainable=False)

            conv = tf.nn.conv2d(self.pool2, weights, [1, 1, 1, 1], padding='SAME')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_1 = tf.nn.relu(out, name=scope)
            self.parameters += [weights, biases]
            
         # conv3_2
        with tf.name_scope('conv3_2') as scope:
            weights = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,
                                                     stddev=1e-1), trainable=False)
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                   trainable=False)
            conv = tf.nn.conv2d(self.conv3_1, weights, [1, 1, 1, 1], padding='SAME')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_2 = tf.nn.relu(out, name=scope)
            self.parameters += [weights, biases]

        # conv3_3
        with tf.name_scope('conv3_3') as scope:
            weights = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,
                                                     stddev=1e-1),  trainable=False)
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                   trainable=False)

            conv = tf.nn.conv2d(self.conv3_2, weights, [1, 1, 1, 1], padding='SAME')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_3 = tf.nn.relu(out, name=scope)
            self.parameters += [weights, biases]

        # pool3
        self.pool3 = tf.nn.max_pool(self.conv3_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool3')
        
         # conv4_1
        with tf.name_scope('conv4_1') as scope:
            weights = tf.Variable(tf.truncated_normal([3, 3, 256, 512], dtype=tf.float32,
                                                     stddev=1e-1), trainable=False)
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                  trainable=False)

            conv = tf.nn.conv2d(self.pool3, weights, [1, 1, 1, 1], padding='SAME')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_1 = tf.nn.relu(out, name=scope)
            self.parameters += [weights, biases]

        # conv4_2
        with tf.name_scope('conv4_2') as scope:
            weights = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), trainable=False)
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                  trainable=False)
            conv = tf.nn.conv2d(self.conv4_1, weights, [1, 1, 1, 1], padding='SAME')
           
            out = tf.nn.bias_add(conv, biases)
            self.conv4_2 = tf.nn.relu(out, name=scope)
            self.parameters += [weights, biases]

        # conv4_3
        with tf.name_scope('conv4_3') as scope:
            weights = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), trainable=False)
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                  trainable=False)
            conv = tf.nn.conv2d(self.conv4_2, weights, [1, 1, 1, 1], padding='SAME')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_3 = tf.nn.relu(out, name=scope)
            self.parameters += [weights, biases]

        # pool4
        self.pool4 = tf.nn.max_pool(self.conv4_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool4')
        
        # conv5_1
        with tf.name_scope('conv5_1') as scope:
            weights = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1),  trainable=False)
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                  trainable=False)
            conv = tf.nn.conv2d(self.pool4, weights, [1, 1, 1, 1], padding='SAME')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_1 = tf.nn.relu(out, name=scope)
            self.parameters += [weights, biases]

        # conv5_2
        with tf.name_scope('conv5_2') as scope:
            weights = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), trainable=False)
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                  trainable=False)
            conv = tf.nn.conv2d(self.conv5_1, weights, [1, 1, 1, 1], padding='SAME')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_2 = tf.nn.relu(out, name=scope)
            self.parameters += [weights, biases]

        # conv5_3
        with tf.name_scope('conv5_3') as scope:
            weights = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), trainable=False)
            conv = tf.nn.conv2d(self.conv5_2, weights, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                  trainable=False)
            out = tf.nn.bias_add(conv, biases)
            self.conv5_3 = tf.nn.relu(out, name=scope)
            self.parameters += [weights, biases]
        
        print('Shape of conv5_3', self.conv5_3.get_shape())
        self.phi_I = tf.einsum('ijkm,ijkn->imn', self.conv5_3, self.conv5_3)
        print('Shape of phi_I after einsum', self.phi_I.get_shape())
        
        self.phi_I = tf.reshape(self.phi_I,[-1,512*512])
        print('Shape of phi_I after reshape', self.phi_I.get_shape())

        self.phi_I = tf.divide(self.phi_I,784.0)  
        print('Shape of phi_I after division', self.phi_I.get_shape())

        self.y_ssqrt = tf.multiply(tf.sign(self.phi_I),tf.sqrt(tf.abs(self.phi_I)+1e-12))
        print('Shape of y_ssqrt', self.y_ssqrt.get_shape())

        self.z_l2 = tf.nn.l2_normalize(self.y_ssqrt, axis=1)
        print('Shape of z_l2', self.z_l2.get_shape())
    
    def fc_layers(self):
            
        with tf.name_scope('fc-new') as scope:
            fc3w = tf.get_variable('weights', [512*512, 129], initializer=tf.contrib.layers.xavier_initializer(),
                                   trainable=True)
            fc3b = tf.get_variable("b", [129], initializer=tf.constant_initializer(0.1), trainable=True)
            self.fc3l = tf.nn.bias_add(tf.matmul(self.z_l2, fc3w), fc3b)
            self.last_layer_parameters += [fc3w, fc3b]
            self.parameters += [fc3w, fc3b]
    
    def load_weights(self, sess):
        weights = np.load(self.weight_file)
        keys = sorted(weights.keys())
        for i, k in enumerate(keys):
            removed_layer_variables = ['fc6_W','fc6_b','fc7_W','fc7_b','fc8_W','fc8_b']
            if not k in removed_layer_variables:
                print(k)
                print("",i, k, np.shape(weights[k]))
                sess.run(self.parameters[i].assign(weights[k]))

In [5]:
sess = tf.Session()
#sess = tf.InteractiveSession()
#with tf.device('/gpu:0'):
imgs = tf.placeholder(tf.float32, [None, 112, 112, 3])
target = tf.placeholder("float", [None, 129])
#print 'Creating graph'
vgg = vgg16_train_last(imgs, 'vgg16_weights.npz', sess)
print('VGG network created')

Shape of conv5_3 (?, 7, 7, 512)
Shape of phi_I after einsum (?, 512, 512)
Shape of phi_I after reshape (?, 262144)
Shape of phi_I after division (?, 262144)
Shape of y_ssqrt (?, 262144)
Shape of z_l2 (?, 262144)
VGG network created


In [6]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=vgg.fc3l, labels=target))
learning_rate_wft = tf.placeholder(tf.float32, shape=[])
learning_rate_woft = tf.placeholder(tf.float32, shape=[])
optimizer = tf.train.MomentumOptimizer(learning_rate=0.9, momentum=0.9).minimize(loss)
correct_prediction = tf.equal(tf.argmax(vgg.fc3l,1), tf.argmax(target,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
num_correct_preds = tf.reduce_sum(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
vgg.load_weights(sess)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

conv1_1_W
 0 conv1_1_W (3, 3, 3, 64)
conv1_1_b
 1 conv1_1_b (64,)
conv1_2_W
 2 conv1_2_W (3, 3, 64, 64)
conv1_2_b
 3 conv1_2_b (64,)
conv2_1_W
 4 conv2_1_W (3, 3, 64, 128)
conv2_1_b
 5 conv2_1_b (128,)
conv2_2_W
 6 conv2_2_W (3, 3, 128, 128)
conv2_2_b
 7 conv2_2_b (128,)
conv3_1_W
 8 conv3_1_W (3, 3, 128, 256)
conv3_1_b
 9 conv3_1_b (256,)
conv3_2_W
 10 conv3_2_W (3, 3, 256, 256)
conv3_2_b
 11 conv3_2_b (256,)
conv3_3_W
 12 conv3_3_W (3, 3, 256, 256)
conv3_3_b
 13 conv3_3_b (256,)
conv4_1_W
 14 conv4_1_W (3, 3, 256, 512)
conv4_1_b
 15 conv4_1_b (512,)
conv4_2_W
 16 conv4_2_W (3, 3, 512, 512)
conv4_2_b
 17 conv4_2_b (512,)
conv4_3_W
 18 conv4_3_W (3, 3, 512, 512)
conv4_3_b
 19 conv4_3_b (512,)
conv5_1_W
 20 conv5_1_W (3, 3, 512, 512)
conv5_1_b
 21 conv5_1_b (512,)
conv5_2_W
 22 conv5_2_W (3, 3, 512, 5

### Train Model

In [7]:
batch_size = 32
lr = 1.0
base_lr = 1.0

for epoch in range(5):
    avg_cost = 0.
    total_batch = int(len(X_train)/batch_size)
    X_train, Y_train = shuffle(X_train, Y_train)
    
    for i in range(total_batch):
        batch_xs, batch_ys = X_train[i*batch_size:i*batch_size+batch_size], Y_train[i*batch_size:i*batch_size+batch_size]
        batch_xs = random_flip_right_to_left(batch_xs)
        start = time.time()
        sess.run(optimizer, feed_dict={imgs: batch_xs, target: batch_ys})
        print('Last layer training, time to run optimizer for batch size:', batch_size,'is --> ',time.time()-start,'seconds')
        cost = sess.run(loss, feed_dict={imgs: batch_xs, target: batch_ys})
        print("Epoch:", '%03d' % (epoch+1), "Step:", '%03d' % i,"Loss:", str(cost))
        print("Training Accuracy -->", sess.run(accuracy,feed_dict={imgs: batch_xs, target: batch_ys}))
    
    val_batch_size = 64
    total_val_count = len(X_val)
    correct_val_count = 0
    val_loss = 0.0
    total_val_batch = int(total_val_count/val_batch_size)
    for i in range(total_val_batch):
        batch_val_x, batch_val_y = X_val[i*val_batch_size:i*val_batch_size+val_batch_size], Y_val[i*val_batch_size:i*val_batch_size+val_batch_size]
        val_loss += sess.run(loss, feed_dict={imgs: batch_val_x, target: batch_val_y})

        pred = sess.run(num_correct_preds, feed_dict = {imgs: batch_val_x, target: batch_val_y})
        correct_val_count+=pred
    
    print("##############################")
    print("Validation Loss -->", val_loss)
    print("correct_val_count, total_val_count", correct_val_count, total_val_count)
    print("Validation Data Accuracy -->", 100.0*correct_val_count/(1.0*total_val_count))
    print("##############################")

Last layer training, time to run optimizer for batch size: 32 is -->  3.4613072872161865 seconds
Epoch: 001 Step: 000 Loss: 4.8098063
Training Accuracy --> 0.0625
Last layer training, time to run optimizer for batch size: 32 is -->  2.965406656265259 seconds
Epoch: 001 Step: 001 Loss: 4.8075533
Training Accuracy --> 0.0625
Last layer training, time to run optimizer for batch size: 32 is -->  2.982403516769409 seconds
Epoch: 001 Step: 002 Loss: 4.8091536
Training Accuracy --> 0.03125
Last layer training, time to run optimizer for batch size: 32 is -->  2.9394121170043945 seconds
Epoch: 001 Step: 003 Loss: 4.7831
Training Accuracy --> 0.03125
Last layer training, time to run optimizer for batch size: 32 is -->  3.0563886165618896 seconds
Epoch: 001 Step: 004 Loss: 4.8032656
Training Accuracy --> 0.0625
Last layer training, time to run optimizer for batch size: 32 is -->  2.9214158058166504 seconds
Epoch: 001 Step: 005 Loss: 4.7932224
Training Accuracy --> 0.03125
Last layer training, tim

Epoch: 004 Step: 002 Loss: 3.8971255
Training Accuracy --> 0.53125
Last layer training, time to run optimizer for batch size: 32 is -->  3.1010000705718994 seconds
Epoch: 004 Step: 003 Loss: 4.019802
Training Accuracy --> 0.34375
Last layer training, time to run optimizer for batch size: 32 is -->  2.989000082015991 seconds
Epoch: 004 Step: 004 Loss: 3.9846916
Training Accuracy --> 0.34375
Last layer training, time to run optimizer for batch size: 32 is -->  2.9049999713897705 seconds
Epoch: 004 Step: 005 Loss: 4.0344834
Training Accuracy --> 0.375
Last layer training, time to run optimizer for batch size: 32 is -->  2.886000156402588 seconds
Epoch: 004 Step: 006 Loss: 3.7613595
Training Accuracy --> 0.46875
Last layer training, time to run optimizer for batch size: 32 is -->  2.9010000228881836 seconds
Epoch: 004 Step: 007 Loss: 3.912314
Training Accuracy --> 0.3125
Last layer training, time to run optimizer for batch size: 32 is -->  2.9100000858306885 seconds
Epoch: 004 Step: 008 Lo

### Test Model

In [8]:

total_test_count = len(X_test)
correct_test_count = 0
test_batch_size = 10
total_test_batch = int(total_test_count/test_batch_size)
for i in range(total_test_batch):
    batch_test_x, batch_test_y = X_test[i*test_batch_size:i*test_batch_size+test_batch_size], Y_test[i*test_batch_size:i*test_batch_size+test_batch_size]
    pred = sess.run(num_correct_preds, feed_dict = {imgs: batch_test_x, target: batch_test_y})
    correct_test_count+=pred

print("##############################")
print("correct_test_count, total_test_count", correct_test_count, total_test_count)
print("Test Data Accuracy -->", 100.0*correct_test_count/(1.0*total_test_count))
print("##############################")

##############################
correct_test_count, total_test_count 120.0 203
Test Data Accuracy --> 59.11330049261084
##############################


### Save Last Layer Weights

In [9]:
last_layer_weights = []
for v in vgg.parameters:
    print(v)
    if v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
        print('Printing Trainable Variables :', sess.run(v).shape)
        last_layer_weights.append(sess.run(v))
np.savez('last_layers_epoch_5.npz',last_layer_weights)
print("Last layer weights saved")

<tf.Variable 'conv1_1/Variable:0' shape=(3, 3, 3, 64) dtype=float32_ref>
<tf.Variable 'conv1_1/Variable_1:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'conv1_2/Variable:0' shape=(3, 3, 64, 64) dtype=float32_ref>
<tf.Variable 'conv1_2/Variable_1:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'conv2_1/Variable:0' shape=(3, 3, 64, 128) dtype=float32_ref>
<tf.Variable 'conv2_1/Variable_1:0' shape=(128,) dtype=float32_ref>
<tf.Variable 'conv2_2/Variable:0' shape=(3, 3, 128, 128) dtype=float32_ref>
<tf.Variable 'conv2_2/Variable_1:0' shape=(128,) dtype=float32_ref>
<tf.Variable 'conv3_1/Variable:0' shape=(3, 3, 128, 256) dtype=float32_ref>
<tf.Variable 'conv3_1/Variable_1:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'conv3_2/Variable:0' shape=(3, 3, 256, 256) dtype=float32_ref>
<tf.Variable 'conv3_2/Variable_1:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'conv3_3/Variable:0' shape=(3, 3, 256, 256) dtype=float32_ref>
<tf.Variable 'conv3_3/Variable_1:0' shape=(256,) dtype=float32_ref>


### VGG16_finetuning

In [4]:
class vgg16_finetuning:
    def __init__(self, imgs, weights=None, sess=None):
        self.images=imgs
        self.imgs = imgs
        self.last_layer_parameters = []     ## Parameters in this list will be optimized when only last layer is being trained 
        self.parameters = []                ## Parameters in this list will be optimized when whole BCNN network is finetuned
        self.convlayers()                   ## Create Convolutional layers
        self.fc_layers()                    ## Create Fully connected layer
        self.weight_file = weights          


    def convlayers(self):
        
        # conv1_1
        with tf.variable_scope("conv1_1"):
            weights = tf.get_variable("W", [3,3,3,64], initializer=tf.contrib.layers.xavier_initializer(), trainable=True)
             # Create variable named "biases".
            biases = tf.get_variable("b", [64], initializer=tf.constant_initializer(0.1), trainable=True)
            conv = tf.nn.conv2d(self.images, weights, strides=[1, 1, 1, 1], padding='SAME')
            self.conv1_1 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]


        # conv1_2
        with tf.variable_scope("conv1_2"):
            weights = tf.get_variable("W", [3,3,64,64], initializer=tf.contrib.layers.xavier_initializer(), trainable=True)
             # Create variable named "biases".
            biases = tf.get_variable("b", [64], initializer=tf.constant_initializer(0.1), trainable=True)
            conv = tf.nn.conv2d(self.conv1_1, weights, strides=[1, 1, 1, 1], padding='SAME')
            self.conv1_2 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]

        # pool1
        self.pool1 = tf.nn.max_pool(self.conv1_2,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool1')

        # conv2_1
        with tf.variable_scope("conv2_1"):
            weights = tf.get_variable("W", [3,3,64,128], initializer=tf.contrib.layers.xavier_initializer(), trainable=True)
             # Create variable named "biases".
            biases = tf.get_variable("b", [128], initializer=tf.constant_initializer(0.1), trainable=True)
            conv = tf.nn.conv2d(self.pool1, weights, strides=[1, 1, 1, 1], padding='SAME')
            self.conv2_1 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]



        # conv2_2
        with tf.variable_scope("conv2_2"):
            weights = tf.get_variable("W", [3,3,128,128], initializer=tf.contrib.layers.xavier_initializer(), trainable=True)
             # Create variable named "biases".
            biases = tf.get_variable("b", [128], initializer=tf.constant_initializer(0.1), trainable=True)
            conv = tf.nn.conv2d(self.conv2_1, weights, strides=[1, 1, 1, 1], padding='SAME')
            self.conv2_2 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]


        # pool2
        self.pool2 = tf.nn.max_pool(self.conv2_2,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool2')

        # conv3_1
        with tf.variable_scope("conv3_1"):
            weights = tf.get_variable("W", [3,3,128,256], initializer=tf.contrib.layers.xavier_initializer(), trainable=True)
             # Create variable named "biases".
            biases = tf.get_variable("b", [256], initializer=tf.constant_initializer(0.1), trainable=True)
            conv = tf.nn.conv2d(self.pool2, weights, strides=[1, 1, 1, 1], padding='SAME')
            self.conv3_1 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]


        # conv3_2
        with tf.variable_scope("conv3_2"):
            weights = tf.get_variable("W", [3,3,256,256], initializer=tf.contrib.layers.xavier_initializer(), trainable=True)
             # Create variable named "biases".
            biases = tf.get_variable("b", [256], initializer=tf.constant_initializer(0.1), trainable=True)
            conv = tf.nn.conv2d(self.conv3_1, weights, strides=[1, 1, 1, 1], padding='SAME')
            self.conv3_2 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]

        # conv3_3
        with tf.variable_scope("conv3_3"):
            weights = tf.get_variable("W", [3,3,256,256], initializer=tf.contrib.layers.xavier_initializer(), trainable=True)
             # Create variable named "biases".
            biases = tf.get_variable("b", [256], initializer=tf.constant_initializer(0.1), trainable=True)
            conv = tf.nn.conv2d(self.conv3_2, weights, strides=[1, 1, 1, 1], padding='SAME')
            self.conv3_3 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]


        # pool3
        self.pool3 = tf.nn.max_pool(self.conv3_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool3')

        # conv4_1
        with tf.variable_scope("conv4_1"):
            weights = tf.get_variable("W", [3,3,256,512], initializer=tf.contrib.layers.xavier_initializer(), trainable=True)
             # Create variable named "biases".
            biases = tf.get_variable("b", [512], initializer=tf.constant_initializer(0.1), trainable=True)
            conv = tf.nn.conv2d(self.pool3, weights, strides=[1, 1, 1, 1], padding='SAME')
            self.conv4_1 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]


        # conv4_2
        with tf.variable_scope("conv4_2"):
            weights = tf.get_variable("W", [3,3,512,512], initializer=tf.contrib.layers.xavier_initializer(), trainable=True)
             # Create variable named "biases".
            biases = tf.get_variable("b", [512], initializer=tf.constant_initializer(0.1), trainable=True)
            conv = tf.nn.conv2d(self.conv4_1, weights, strides=[1, 1, 1, 1], padding='SAME')
            self.conv4_2 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]


        # conv4_3
        with tf.variable_scope("conv4_3"):
            weights = tf.get_variable("W", [3,3,512,512], initializer=tf.contrib.layers.xavier_initializer(), trainable=True)
             # Create variable named "biases".
            biases = tf.get_variable("b", [512], initializer=tf.constant_initializer(0.1), trainable=True)
            conv = tf.nn.conv2d(self.conv4_2, weights, strides=[1, 1, 1, 1], padding='SAME')
            self.conv4_3 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]

        # pool4
        self.pool4 = tf.nn.max_pool(self.conv4_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool4')

        # conv5_1
        with tf.variable_scope("conv5_1"):
            weights = tf.get_variable("W", [3,3,512,512], initializer=tf.contrib.layers.xavier_initializer(), trainable=True)
             # Create variable named "biases".
            biases = tf.get_variable("b", [512], initializer=tf.constant_initializer(0.1), trainable=True)
            conv = tf.nn.conv2d(self.pool4, weights, strides=[1, 1, 1, 1], padding='SAME')
            self.conv5_1 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]


        # conv5_2
        with tf.variable_scope("conv5_2"):
            weights = tf.get_variable("W", [3,3,512,512], initializer=tf.contrib.layers.xavier_initializer(), trainable=True)
             # Create variable named "biases".
            biases = tf.get_variable("b", [512], initializer=tf.constant_initializer(0.1), trainable=True)
            conv = tf.nn.conv2d(self.conv5_1, weights, strides=[1, 1, 1, 1], padding='SAME')
            self.conv5_2 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]
            

        # conv5_3
        with tf.variable_scope("conv5_3"):
            weights = tf.get_variable("W", [3,3,512,512], initializer=tf.contrib.layers.xavier_initializer(), trainable=True)
             # Create variable named "biases".
            biases = tf.get_variable("b", [512], initializer=tf.constant_initializer(0.1), trainable=True)
            conv = tf.nn.conv2d(self.conv5_2, weights, strides=[1, 1, 1, 1], padding='SAME')
            self.conv5_3 = tf.nn.relu(conv + biases)

            self.parameters += [weights, biases]

        #self.conv5_3 = tf.transpose(self.conv5_3, perm=[0,3,1,2])       
        #self.conv5_3 = tf.reshape(self.conv5_3,[-1,512,36])          
        #conv5_3_T = tf.transpose(self.conv5_3, perm=[0,2,1])     
        #self.phi_I = tf.matmul(self.conv5_3, conv5_3_T)        

        print('Shape of conv5_3', self.conv5_3.get_shape())
        self.phi_I = tf.einsum('ijkm,ijkn->imn', self.conv5_3, self.conv5_3)
        print('Shape of phi_I after einsum', self.phi_I.get_shape())
        self.phi_I = tf.reshape(self.phi_I,[-1,512*512])            
        
        print('Shape of phi_I after reshape', self.phi_I.get_shape())

        self.phi_I = tf.divide(self.phi_I,784.0)  
        
        print('Shape of phi_I after division', self.phi_I.get_shape())  

        self.y_ssqrt = tf.multiply(tf.sign(self.phi_I),tf.sqrt(tf.abs(self.phi_I)+1e-12))       
        print('Shape of y_ssqrt', self.y_ssqrt.get_shape())

        self.z_l2 = tf.nn.l2_normalize(self.y_ssqrt, dim=1)     
        print('Shape of z_l2', self.z_l2.get_shape())




    def fc_layers(self):

        with tf.variable_scope('fc-new') as scope:
            fc3w = tf.get_variable('W', [512*512, 129], initializer=tf.contrib.layers.xavier_initializer(), trainable=True)
            #fc3b = tf.Variable(tf.constant(1.0, shape=[100], dtype=tf.float32), name='biases', trainable=True)
            fc3b = tf.get_variable("b", [129], initializer=tf.constant_initializer(0.1), trainable=True)
            self.fc3l = tf.nn.bias_add(tf.matmul(self.z_l2, fc3w), fc3b)
            self.last_layer_parameters += [fc3w, fc3b]

    def load_initial_weights(self, session):

        '''weight_dict contains weigths of VGG16 layers'''
        weights_dict = np.load(self.weight_file, encoding = 'bytes')

        
        '''Loop over all layer names stored in the weights dict
           Load only conv-layers. Skip fc-layers in VGG16'''
        vgg_layers = ['conv1_1','conv1_2','conv2_1','conv2_2','conv3_1','conv3_2','conv3_3','conv4_1','conv4_2','conv4_3','conv5_1','conv5_2','conv5_3']
        
        for op_name in vgg_layers:
            with tf.variable_scope(op_name, reuse = True):
                
              # Loop over list of weights/biases and assign them to their corresponding tf variable
                # Biases
              
              var = tf.get_variable('b', trainable = True)
              print('Adding weights to',var.name)
              session.run(var.assign(weights_dict[op_name+'_b']))
                  
            # Weights
              var = tf.get_variable('W', trainable = True)
              print('Adding weights to',var.name)
              session.run(var.assign(weights_dict[op_name+'_W']))

        with tf.variable_scope('fc-new', reuse = True):
            '''
            Load fc-layer weights trained in the first step. 
            Use file .py to train last layer
            '''
            last_layer_weights = np.load('last_layers_epoch_5.npz')
            print('Last layer weights: last_layers_epoch_5.npz')
            var = tf.get_variable('W', trainable = True)
            print('Adding weights to',var.name)
            session.run(var.assign(last_layer_weights['arr_0'][0]))
            var = tf.get_variable('b', trainable = True)
            print('Adding weights to',var.name)
            session.run(var.assign(last_layer_weights['arr_0'][1]))

In [5]:
sess = tf.Session()     ## Start session to create training graph

imgs = tf.placeholder(tf.float32, [None, 112, 112, 3])
target = tf.placeholder("float", [None, 129])
#print 'Creating graph'
vgg = vgg16_finetuning(imgs, 'vgg16_weights.npz', sess)

print('VGG network created')
    
# Defining other ops using Tensorflow
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=vgg.fc3l, labels=target))
print([_.name for _ in vgg.parameters])
optimizer = tf.train.MomentumOptimizer(learning_rate=0.001, momentum=0.9).minimize(loss)
check_op = tf.add_check_numerics_ops()
correct_prediction = tf.equal(tf.argmax(vgg.fc3l,1), tf.argmax(target,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
num_correct_preds = tf.reduce_sum(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
vgg.load_initial_weights(sess)
print([_.name for _ in vgg.parameters])

Shape of conv5_3 (?, 7, 7, 512)
Shape of phi_I after einsum (?, 512, 512)
Shape of phi_I after reshape (?, 262144)
Shape of phi_I after division (?, 262144)
Shape of y_ssqrt (?, 262144)
Instructions for updating:
dim is deprecated, use axis instead
Shape of z_l2 (?, 262144)
VGG network created
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

['conv1_1/W:0', 'conv1_1/b:0', 'conv1_2/W:0', 'conv1_2/b:0', 'conv2_1/W:0', 'conv2_1/b:0', 'conv2_2/W:0', 'conv2_2/b:0', 'conv3_1/W:0', 'conv3_1/b:0', 'conv3_2/W:0', 'conv3_2/b:0', 'conv3_3/W:0', 'conv3_3/b:0', 'conv4_1/W:0', 'conv4_1/b:0', 'conv4_2/W:0', 'conv4_2/b:0', 'conv4_3/W:0', 'conv4_3/b:0', 'conv5_1/W:0', 'conv5_1/b:0', 'conv5_2/W:0', 'conv5_2/b:0', 'conv5_3/W:0', 'conv5_3/b:0']
Adding weights to conv1_1/b:0
Adding weights to conv1_1/W:0
Adding weights to conv1_2/b:0
Adding weights to conv1_2/W:0
Addin

### Train finetuning Model

In [6]:
for v in tf.trainable_variables():
        print("Trainable variables", v)
        
batch_size = 16
lr = 0.001
finetune_step = -1
validation_accuracy_buffer = []

Trainable variables <tf.Variable 'conv1_1/W:0' shape=(3, 3, 3, 64) dtype=float32_ref>
Trainable variables <tf.Variable 'conv1_1/b:0' shape=(64,) dtype=float32_ref>
Trainable variables <tf.Variable 'conv1_2/W:0' shape=(3, 3, 64, 64) dtype=float32_ref>
Trainable variables <tf.Variable 'conv1_2/b:0' shape=(64,) dtype=float32_ref>
Trainable variables <tf.Variable 'conv2_1/W:0' shape=(3, 3, 64, 128) dtype=float32_ref>
Trainable variables <tf.Variable 'conv2_1/b:0' shape=(128,) dtype=float32_ref>
Trainable variables <tf.Variable 'conv2_2/W:0' shape=(3, 3, 128, 128) dtype=float32_ref>
Trainable variables <tf.Variable 'conv2_2/b:0' shape=(128,) dtype=float32_ref>
Trainable variables <tf.Variable 'conv3_1/W:0' shape=(3, 3, 128, 256) dtype=float32_ref>
Trainable variables <tf.Variable 'conv3_1/b:0' shape=(256,) dtype=float32_ref>
Trainable variables <tf.Variable 'conv3_2/W:0' shape=(3, 3, 256, 256) dtype=float32_ref>
Trainable variables <tf.Variable 'conv3_2/b:0' shape=(256,) dtype=float32_ref>


In [7]:
for epoch in range(5):
    avg_cost = 0.
    total_batch = int(len(X_train)/batch_size)
    X_train, Y_train = shuffle(X_train, Y_train)


    for i in range(total_batch):
        batch_xs, batch_ys = X_train[i*batch_size:i*batch_size+batch_size], Y_train[i*batch_size:i*batch_size+batch_size]
        batch_xs = random_flip_right_to_left(batch_xs)    
        start = time.time()
        sess.run([optimizer,check_op], feed_dict={imgs: batch_xs, target: batch_ys})
        if i%20==0:
            print('Full BCNN finetuning, time to run optimizer for batch size 16:',time.time()-start,'seconds')


        cost = sess.run(loss, feed_dict={imgs: batch_xs, target: batch_ys})
            
        if i % 20 == 0:
            print ('Learning rate: ', (str(lr)))
            if epoch <= finetune_step:
                print("Training last layer of BCNN_DD")
            else:
                print("Fine tuning all BCNN_DD")

            print("Epoch:", '%03d' % (epoch+1), "Step:", '%03d' % i,"Loss:", str(cost))
            print("Training Accuracy -->", accuracy.eval(feed_dict={imgs: batch_xs, target: batch_ys}, session=sess))
    val_batch_size = 10
    total_val_count = len(X_val)
    correct_val_count = 0
    val_loss = 0.0
    total_val_batch = int(total_val_count/val_batch_size)
    for i in range(total_val_batch):
        batch_val_x, batch_val_y = X_val[i*val_batch_size:i*val_batch_size+val_batch_size], Y_val[i*val_batch_size:i*val_batch_size+val_batch_size]
        val_loss += sess.run(loss, feed_dict={imgs: batch_val_x, target: batch_val_y})

        pred = sess.run(num_correct_preds, feed_dict = {imgs: batch_val_x, target: batch_val_y})
        correct_val_count+=pred

    print("##############################")
    print("Validation Loss -->", val_loss)
    print("correct_val_count, total_val_count", correct_val_count, total_val_count)
    print("Validation Data Accuracy -->", 100.0*correct_val_count/(1.0*total_val_count))
    print("##############################")

    if epoch>40:

        validation_accuracy_buffer.append(100.0*correct_val_count/(1.0*total_val_count))
        ## Check if the validation accuracy has stopped increasing
        if len(validation_accuracy_buffer)>10:
            index_of_max_val_acc = np.argmax(validation_accuracy_buffer)
            if index_of_max_val_acc==0:
                break
            else:
                del validation_accuracy_buffer[0]


Full BCNN finetuning, time to run optimizer for batch size 16: 11.992999792098999 seconds
Learning rate:  0.001
Fine tuning all BCNN_DD
Epoch: 001 Step: 000 Loss: 3.2198853
Training Accuracy --> 0.8125
Full BCNN finetuning, time to run optimizer for batch size 16: 11.009000301361084 seconds
Learning rate:  0.001
Fine tuning all BCNN_DD
Epoch: 001 Step: 020 Loss: 3.6173303
Training Accuracy --> 0.5
##############################
Validation Loss --> 68.63584733009338
correct_val_count, total_val_count 118.0 203
Validation Data Accuracy --> 58.12807881773399
##############################
Full BCNN finetuning, time to run optimizer for batch size 16: 10.61299991607666 seconds
Learning rate:  0.001
Fine tuning all BCNN_DD
Epoch: 002 Step: 000 Loss: 3.4130616
Training Accuracy --> 0.75
Full BCNN finetuning, time to run optimizer for batch size 16: 10.877000093460083 seconds
Learning rate:  0.001
Fine tuning all BCNN_DD
Epoch: 002 Step: 020 Loss: 3.2949257
Training Accuracy --> 0.6875
######

In [8]:
full_layer_weights = []
for v in vgg.parameters:
    print(v)
    if v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
        print('Printing Trainable Variables :', sess.run(v).shape)
        full_layer_weights.append(sess.run(v))
for v in vgg.last_layer_parameters:
    print(v)
    if v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
        print('Printing Trainable Variables :', sess.run(v).shape)
        full_layer_weights.append(sess.run(v))
        
np.savez('full_layers_epoch_5.npz',full_layer_weights)
print("full layer weights saved")

<tf.Variable 'conv1_1/W:0' shape=(3, 3, 3, 64) dtype=float32_ref>
Printing Trainable Variables : (3, 3, 3, 64)
<tf.Variable 'conv1_1/b:0' shape=(64,) dtype=float32_ref>
Printing Trainable Variables : (64,)
<tf.Variable 'conv1_2/W:0' shape=(3, 3, 64, 64) dtype=float32_ref>
Printing Trainable Variables : (3, 3, 64, 64)
<tf.Variable 'conv1_2/b:0' shape=(64,) dtype=float32_ref>
Printing Trainable Variables : (64,)
<tf.Variable 'conv2_1/W:0' shape=(3, 3, 64, 128) dtype=float32_ref>
Printing Trainable Variables : (3, 3, 64, 128)
<tf.Variable 'conv2_1/b:0' shape=(128,) dtype=float32_ref>
Printing Trainable Variables : (128,)
<tf.Variable 'conv2_2/W:0' shape=(3, 3, 128, 128) dtype=float32_ref>
Printing Trainable Variables : (3, 3, 128, 128)
<tf.Variable 'conv2_2/b:0' shape=(128,) dtype=float32_ref>
Printing Trainable Variables : (128,)
<tf.Variable 'conv3_1/W:0' shape=(3, 3, 128, 256) dtype=float32_ref>
Printing Trainable Variables : (3, 3, 128, 256)
<tf.Variable 'conv3_1/b:0' shape=(256,) dty

In [50]:
test_data = h5py.File('test_96.h5', 'r')
X_test, Y_test = test_data['X'], test_data['Y']
print(X_test.shape)
print(Y_test.shape)

(12800, 96, 96, 3)
(12800,)


In [51]:
prediction = tf.argmax(vgg.fc3l,1)

In [52]:
total_test_count = len(X_test)
correct_test_count = 0
test_batch_size = 10
total_test_batch = int(total_test_count/test_batch_size)
#for i in range(total_test_batch):
with open('pred.txt', 'w') as text_file:
    for i in range(total_test_batch):
        batch_test_x, x_id = X_test[i*test_batch_size:i*test_batch_size+test_batch_size], Y_test[i*test_batch_size:i*test_batch_size+test_batch_size]
        pred = sess.run(prediction, feed_dict = {imgs: batch_test_x})
        print(pred)
        print(x_id[:])
        for i in range(len(x_id)):
            text_file.write('{0},{1}\n'.format(int(x_id[i])+1,pred[i]))

[ 27 117  83  63  86  99  74  19   7  83]
[  1.00000000e+00   1.00000000e+01   1.00000000e+02   1.00000000e+03
   1.00000000e+04   1.00010000e+04   1.00020000e+04   1.00030000e+04
   1.00040000e+04   1.00050000e+04]
[ 31  80 106 122 117 106  27  96  40 122]
[ 10006.  10007.  10008.  10009.   1001.  10010.  10011.  10012.  10013.
  10014.]
[ 35  78  43  62  94  58  26 101  43  13]
[ 10015.  10016.  10017.  10018.  10019.   1002.  10020.  10021.  10022.
  10023.]
[113   2 111   6  54   8  45  28  94  22]
[ 10024.  10025.  10026.  10027.  10028.  10029.   1003.  10030.  10031.
  10032.]
[ 36   9  17  27  68  29  40 121  31  85]
[ 10033.  10034.  10035.  10036.  10037.  10038.  10039.   1004.  10040.
  10041.]
[ 78  11   2  72  60  90 106  88  14  31]
[ 10042.  10043.  10044.  10045.  10046.  10047.  10048.  10049.   1005.
  10050.]
[ 13  45  74 116  83  67 101 123  79  33]
[ 10051.  10052.  10053.  10054.  10055.  10056.  10057.  10058.  10059.
   1006.]
[  4  56  35  86  37  31  88  29  

[ 64 122 115  83  18  74  37 121  13  53]
[ 10592.  10593.  10594.  10595.  10596.  10597.  10598.  10599.    106.
   1060.]
[  3  98 122  80  45 114 117  88  11  48]
[ 10600.  10601.  10602.  10603.  10604.  10605.  10606.  10607.  10608.
  10609.]
[ 95  56  58 109  63  27  19  31  78 128]
[  1061.  10610.  10611.  10612.  10613.  10614.  10615.  10616.  10617.
  10618.]
[ 27   5   4   4  93  54  86 114  31  45]
[ 10619.   1062.  10620.  10621.  10622.  10623.  10624.  10625.  10626.
  10627.]
[ 53   9  39  63  48  33 111  83  27  28]
[ 10628.  10629.   1063.  10630.  10631.  10632.  10633.  10634.  10635.
  10636.]
[  4  80  35  40  53  78   4 103 114  68]
[ 10637.  10638.  10639.   1064.  10640.  10641.  10642.  10643.  10644.
  10645.]
[104 117  19   3  43   5  85  64 109  58]
[ 10646.  10647.  10648.  10649.   1065.  10650.  10651.  10652.  10653.
  10654.]
[91 29 78 62 89 96 87 94 83 40]
[ 10655.  10656.  10657.  10658.  10659.   1066.  10660.  10661.  10662.
  10663.]
[ 65  20  

[108 122  13  54  28  83   8  79  85 122]
[ 11186.  11187.  11188.  11189.   1119.  11190.  11191.  11192.  11193.
  11194.]
[ 96  37 126  68  67 106  19 106 115  46]
[ 11195.  11196.  11197.  11198.  11199.    112.   1120.  11200.  11201.
  11202.]
[ 27  22   7  43  68  64 117 114 122  14]
[ 11203.  11204.  11205.  11206.  11207.  11208.  11209.   1121.  11210.
  11211.]
[  1 109  54 117  89  20  54  83 123   7]
[ 11212.  11213.  11214.  11215.  11216.  11217.  11218.  11219.   1122.
  11220.]
[  8 101  54  93  44  27  75 109 116  82]
[ 11221.  11222.  11223.  11224.  11225.  11226.  11227.  11228.  11229.
   1123.]
[ 80  96  74  54  31  86 103 121  40  58]
[ 11230.  11231.  11232.  11233.  11234.  11235.  11236.  11237.  11238.
  11239.]
[127   9  90  54 110 110 123  74  89  74]
[  1124.  11240.  11241.  11242.  11243.  11244.  11245.  11246.  11247.
  11248.]
[ 79   8  19  83   4  74  23  23 126  27]
[ 11249.   1125.  11250.  11251.  11252.  11253.  11254.  11255.  11256.
  11257.]


[ 45  89  44  83  67  36   5  96 106  79]
[ 11780.  11781.  11782.  11783.  11784.  11785.  11786.  11787.  11788.
  11789.]
[  1 104 110  83  96 123 121  80   9  67]
[  1179.  11790.  11791.  11792.  11793.  11794.  11795.  11796.  11797.
  11798.]
[117  19 101  45  39  75 122   5 122  88]
[ 11799.    118.   1180.  11800.  11801.  11802.  11803.  11804.  11805.
  11806.]
[ 65  28  87  40  27   4 117  52  95  93]
[ 11807.  11808.  11809.   1181.  11810.  11811.  11812.  11813.  11814.
  11815.]
[  1  19  65   5  85  82  44  39 123  38]
[ 11816.  11817.  11818.  11819.   1182.  11820.  11821.  11822.  11823.
  11824.]
[ 57  44 111  27  39  76  35 110 116  35]
[ 11825.  11826.  11827.  11828.  11829.   1183.  11830.  11831.  11832.
  11833.]
[ 35  19  27  31  86  76   5 109   8 117]
[ 11834.  11835.  11836.  11837.  11838.  11839.   1184.  11840.  11841.
  11842.]
[ 84 123 108  49  36  85  43 111  98  66]
[ 11843.  11844.  11845.  11846.  11847.  11848.  11849.   1185.  11850.
  11851.]


[ 24  79  78  50   3  95 108  86  26  75]
[ 12365.  12366.  12367.  12368.  12369.   1237.  12370.  12371.  12372.
  12373.]
[ 5 38  9 38 27 81 83 79 10 28]
[ 12374.  12375.  12376.  12377.  12378.  12379.   1238.  12380.  12381.
  12382.]
[ 13  84  81 117  85  54  93  98  85  78]
[ 12383.  12384.  12385.  12386.  12387.  12388.  12389.   1239.  12390.
  12391.]
[106 107  62 101  60  81  45   4  75  98]
[ 12392.  12393.  12394.  12395.  12396.  12397.  12398.  12399.    124.
   1240.]
[ 20  95 111 108  73  54   7 122  53 106]
[ 12400.  12401.  12402.  12403.  12404.  12405.  12406.  12407.  12408.
  12409.]
[107  17  94  97  84  95  90 107 101  63]
[  1241.  12410.  12411.  12412.  12413.  12414.  12415.  12416.  12417.
  12418.]
[72 19 14 11 88 29 83 68 28 68]
[ 12419.   1242.  12420.  12421.  12422.  12423.  12424.  12425.  12426.
  12427.]
[123   4  28  83 113  18 122  87 122  37]
[ 12428.  12429.   1243.  12430.  12431.  12432.  12433.  12434.  12435.
  12436.]
[ 54  54  93  79  54

...

[ 50  53 113  28  85  40 104 117 122  46]
[ 7334.  7335.  7336.  7337.  7338.  7339.   734.  7340.  7341.  7342.]
[ 54  38  43  72  80 114  83   9  35  67]
[ 7343.  7344.  7345.  7346.  7347.  7348.  7349.   735.  7350.  7351.]
[  4  49  52  96  54 106  37 127  94  63]
[ 7352.  7353.  7354.  7355.  7356.  7357.  7358.  7359.   736.  7360.]
[ 11  74  68  45  58  79  14 106  89 107]
[ 7361.  7362.  7363.  7364.  7365.  7366.  7367.  7368.  7369.   737.]
[101 108  95 111  72  27  19  68  27   1]
[ 7370.  7371.  7372.  7373.  7374.  7375.  7376.  7377.  7378.  7379.]
[ 13  83   8  65  17 111  35 108  67 114]
[  738.  7380.  7381.  7382.  7383.  7384.  7385.  7386.  7387.  7388.]
[107 107  80  77   9  36  20  85 113   8]
[ 7389.   739.  7390.  7391.  7392.  7393.  7394.  7395.  7396.  7397.]
[ 45 122 107  96  80   6  77 122  49  99]
[ 7398.  7399.    74.   740.  7400.  7401.  7402.  7403.  7404.  7405.]
[102 128 106  74   8  65   7  88  24  46]
[ 7406.  7407.  7408.  7409.   741.  7410.  74

[106   8  62  74  74  83 120  68  89  83]
[ 7992.  7993.  7994.  7995.  7996.  7997.  7998.  7999.     8.    80.]
[74 65 40 19  8 40 83 19 28 20]
[  800.  8000.  8001.  8002.  8003.  8004.  8005.  8006.  8007.  8008.]
[ 74  86  20  97  53  35  19 117 110 116]
[ 8009.   801.  8010.  8011.  8012.  8013.  8014.  8015.  8016.  8017.]
[ 79   4  98   9  78  42  40 104  98  10]
[ 8018.  8019.   802.  8020.  8021.  8022.  8023.  8024.  8025.  8026.]
[106  76 106  54  20  46  63  48 117  79]
[ 8027.  8028.  8029.   803.  8030.  8031.  8032.  8033.  8034.  8035.]
[ 14  20  83  64 107  16 115  58  69  10]
[ 8036.  8037.  8038.  8039.   804.  8040.  8041.  8042.  8043.  8044.]
[124  54   4  89   5  11  85  93  38  85]
[ 8045.  8046.  8047.  8048.  8049.   805.  8050.  8051.  8052.  8053.]
[ 58  35  87  31  27  85  68 123  50 127]
[ 8054.  8055.  8056.  8057.  8058.  8059.   806.  8060.  8061.  8062.]
[ 61  10  63  94  24  98  99 106 108 114]
[ 8063.  8064.  8065.  8066.  8067.  8068.  8069.   807.

[ 80  96 114 109  93  10  88  45  53  94]
[ 8649.   865.  8650.  8651.  8652.  8653.  8654.  8655.  8656.  8657.]
[ 45  79 122 123  39  49  76  28  83  38]
[ 8658.  8659.   866.  8660.  8661.  8662.  8663.  8664.  8665.  8666.]
[107  49 122  28  38  97  63  11 108  98]
[ 8667.  8668.  8669.   867.  8670.  8671.  8672.  8673.  8674.  8675.]
[ 16  64  86 124  83 106 117  19   4  81]
[ 8676.  8677.  8678.  8679.   868.  8680.  8681.  8682.  8683.  8684.]
[124 122 107  83 111   3  87  92  35 115]
[ 8685.  8686.  8687.  8688.  8689.   869.  8690.  8691.  8692.  8693.]
[122  75  78  34  50  86  74  80  21  94]
[ 8694.  8695.  8696.  8697.  8698.  8699.    87.   870.  8700.  8701.]
[ 60 117 107  54 106  58  60   1  19  49]
[ 8702.  8703.  8704.  8705.  8706.  8707.  8708.  8709.   871.  8710.]
[  5  95  86  19  49  76 108  74  58  52]
[ 8711.  8712.  8713.  8714.  8715.  8716.  8717.  8718.  8719.   872.]
[110  24 122   2  80  74  98  35  56 110]
[ 8720.  8721.  8722.  8723.  8724.  8725.  87

[ 78  96  17  87  10  82 114  77 111  42]
[ 9305.  9306.  9307.  9308.  9309.   931.  9310.  9311.  9312.  9313.]
[ 10  74  98  68 111  72  37  36  87 106]
[ 9314.  9315.  9316.  9317.  9318.  9319.   932.  9320.  9321.  9322.]
[122  83  68  74  49  53  83  28  78 109]
[ 9323.  9324.  9325.  9326.  9327.  9328.  9329.   933.  9330.  9331.]
[ 27   5   3 117  56 122  58  95  80  92]
[ 9332.  9333.  9334.  9335.  9336.  9337.  9338.  9339.   934.  9340.]
[ 45  38   1  20 110 104 117 127  97  96]
[ 9341.  9342.  9343.  9344.  9345.  9346.  9347.  9348.  9349.   935.]
[ 62  31  81  27 106 110  83  39  66  54]
[ 9350.  9351.  9352.  9353.  9354.  9355.  9356.  9357.  9358.  9359.]
[ 78  13  53  19  40  20 122  49 122  54]
[  936.  9360.  9361.  9362.  9363.  9364.  9365.  9366.  9367.  9368.]
[29 19  4 56 27 89 20 83 95 68]
[ 9369.   937.  9370.  9371.  9372.  9373.  9374.  9375.  9376.  9377.]
[ 95 127 108  99 103 108  48  75  10 123]
[ 9378.  9379.   938.  9380.  9381.  9382.  9383.  9384.

[123  22  83 124  30  94  85  88  35 124]
[ 9963.  9964.  9965.  9966.  9967.  9968.  9969.   997.  9970.  9971.]
[ 38  74 108   3 123  62  57  65  65  28]
[ 9972.  9973.  9974.  9975.  9976.  9977.  9978.  9979.   998.  9980.]
[ 31  19  62 117  28 122  93  18 111  90]
[ 9981.  9982.  9983.  9984.  9985.  9986.  9987.  9988.  9989.   999.]
[ 83  58 106  67 108  23  53  83   8  83]
[ 9990.  9991.  9992.  9993.  9994.  9995.  9996.  9997.  9998.  9999.]
